# Notebook Contents

- [Imports](#Imports)
- [Data](#Data)
- [Pitcher Overview](#Pitcher-Overview)
    - [Pitch Usage](#Pitch-Usage)
    - [Pitch Usage by Batter Handedness](#Pitch-Usage-by-Batter-Handedness)
    - [Velocity by Pitch Type](#Velocity-by-Pitch-Type)
- [Pitcher Stuff](#Pitcher-Stuff)
    - [Spin Axis](#Spin-Axis)
    - [Velocity & Spin Rate](#Velocity-&-Spin-Rate)
    - [HB & VB Axis](#Horizontal-&-Vertical-Break-Axis)
    - [Release Position](#Release-Postion)
- [Count Breakdown](#Count-Breakdown)
    - [Pitch Usage by Count](#Pitch-Usage-by-Count)
- [Pitch Locations](#Pitch-Locations)
    - [Balls-&Strikes-from-Hitters' Perspective - All-Pitches](#Balls-&-Strikes-from-Hitters'-Perspective--All-Pitches)
- [Heatmaps](#Heatmaps)
    - [Slider Heatmaps](#Slider-Heatmaps)
    - [Fastball Heatmaps](#Fastball-Heatmaps)
    - [Curveball Heatmaps](#Curveball-Heatmaps)
    - [Changeup Heatmaps](#Changeup-Heatmaps)

# Imports

In [1]:
from math import radians
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
from matplotlib.ticker import PercentFormatter

import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
scherzer = pd.read_csv('../data/max-scherzer.csv')
scherzer.drop(columns = ['Unnamed: 0'], inplace = True)

# Font Dictionary
font_title = {
    'size': 14,
    'weight': 'bold',
    'verticalalignment': 'center_baseline',
    'horizontalalignment': 'center'
}

pd.set_option('max_columns', None)
print(scherzer.shape)
scherzer.head(2)

(2821, 90)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,is_strike,pitch_count,plate_-x,pfx_-x,bauer_units,swing_miss
0,FF,2021-09-29,94.3,-3.23,5.43,"Scherzer, Max",571976,453286,triple,hit_into_play,1,Wil Myers triples (2) on a line drive to left ...,R,R,R,LAD,SD,X,8.0,line_drive,0,2,2021,-1.22,17.28,-0.37,3.24,NaN,NaN,605170.0,1,6,Top,96.1,49.21,669257,10.132657,-136.967197,-3.168637,-17.601606,30.075797,-13.539700,3.49,1.6,376.0,98.8,23.0,94.2,2486.0,6.3,632297,453286,669257,571970,607208,621035,608369,572041,666158,605141,54.20,0.446,0.612,1.6,1.0,1.0,2.0,5.0,50,3,4-Seam Fastball,5,5,5,5,6,5,6,5,Infield shift,Standard,230.0,-0.224,1.555,1,0-2,0.37,14.64,26.362672,0
1,SL,2021-09-29,84.8,-3.51,5.33,"Scherzer, Max",571976,453286,NaN,swinging_strike,4,Wil Myers triples (2) on a line drive to left ...,R,R,R,LAD,SD,S,NaN,NaN,0,1,2021,-0.03,6.24,-0.72,2.62,NaN,NaN,605170.0,1,6,Top,NaN,NaN,669257,6.597895,-123.444921,-1.129416,-1.452731,20.644158,-26.818383,3.49,1.6,NaN,NaN,NaN,85.4,2439.0,6.3,632297,453286,669257,571970,607208,621035,608369,572041,666158,605141,54.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,2,Slider,5,5,5,5,5,5,5,5,Infield shift,Standard,183.0,0.000,-0.064,1,0-1,0.72,0.36,28.761792,1
